In [16]:
cd "Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format"

Q:\DATA\SPRLP\EM\Review of Conditionality\Literature\Program approval SRs\text format


In [81]:
import pandas as pd
import numpy as np
import seaborn as sns

#viewing for the first sample
with open('Afganistan 2011 ECF.txt', encoding = 'utf-8') as f:
    list = [line.rstrip('\n') for line in f]

In [82]:
list

['© 2011 International Monetary Fund',
 'November 2, 2011',
 '',
 '2011',
 '',
 'November 2011',
 'IMF Country Report No. 11/330',
 'November 10, 2011',
 '',
 'Islamic Republic of Afghanistan: 2011 Article IV Consultation and Request for a ThreeYear Arrangement Under the Extended Credit Facility—Staff Report; Staff',
 'Supplement–A Joint World Bank/IMF Debt Sustainability Analysis; Staff Statement;',
 'Public Information Notice and Press Release on the Executive Board Discussion; and',
 'Statement by the Executive Director for Islamic Republic of Afghanistan.',
 'Under Article IV of the IMF’s Articles of Agreement, the IMF holds bilateral discussions with',
 'members, usually every year. In the context of a combined discussion of the 2011 Article IV',
 'consultation with Afghanistan and Request for a Three-Year Arrangement Under the Extended Credit',
 'Facility, the following documents have been released and are included in this package:',
 '•',
 '',
 'The staff report for the combined

In [83]:
df=pd.DataFrame({'text':list})

In [84]:
print(df.shape)
df.tail(10)

(15946, 1)


,text
15936,"been strewn with formidable obstacles, includi..."
15937,"governance issues, and low implementation capa..."
15938,"many areas as outlined above, which should not..."
15939,Kabul Bank crisis. The reform agenda in the ye...
15940,"perseverance, flexibility, and sustained donor..."
15941,that has demonstrated its effectiveness in ser...
15942,"appreciate Fund advice and assistance, and are..."
15943,under the new program.
15944,
15945,


In [55]:
sub_df=df.loc[df.text.str.contains('consolidation')]

In [85]:
sub_df['prog']='Afg'

In [98]:
sub_df

,text,prog
512,in early-2009 and the consolidation of earlier...,Afg
7391,strengthening the banking law and encouraging ...,Afg
7397,system’s capacity to absorb losses and encoura...,Afg
9191,consolidation of the fiscal reports and not re...,Afg
15909,"encouraging consolidation of small banks, and ...",Afg


In [99]:
args[[i for i, x in enumerate(df) if df.text.str.contains('consolidation')][0] + 1]

NameError: name 'args' is not defined

AttributeError: 'enumerate' object has no attribute 'list'

ImportError: No module named 'textblob'